In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import torchvision as T
import torch
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import Dataset, DataLoader
import math
import itertools
import os
import sys
root_path = os.path.abspath(os.path.join('../..')) # <- adjust such that root_path always points at the root project dir (i.e. if current file is two folders deep, use '../..'). 
if root_path not in sys.path:
    sys.path.append(root_path)
import database_server.db_utilities as dbu 
import pickle as pkl
from sklearn.decomposition import PCA
from Help_functions import preprocess, game_dict, inputs, club_dict, points_and_co, points_and_co_oppon, data_to_lstm, Sport_pred_2LSTM_1, predict
import random

In [2]:
torch.manual_seed(42069)
random.seed(42069)
np.random.seed(42069)

In [3]:
query_str = """
SELECT ms.*, 
       m.schedule_date, m.schedule_time, m.schedule_round, m.schedule_day,
       w.annual_wages_eur AS annual_wage_team, 
       w.weekly_wages_eur AS weekly_wages_eur,
       w.annual_wages_eur/w.n_players AS annual_wage_player_avg
FROM matchstats ms 
LEFT JOIN matches m ON ms.match_id = m.id
LEFT JOIN teamwages w ON ms.team_id = w.team_id
AND     ms.season_str = w.season_str
ORDER BY m.schedule_date DESC, m.schedule_time DESC; 
"""

df_allinfo = dbu.select_query(query_str)

In [4]:
new_data_test = preprocess(df_allinfo)

In [5]:
scale_df = new_data_test.data_frame.copy()
#scale_df["stad_capac"] = 0
for team in scale_df.team_id.unique():
    max_attend = max(scale_df[(scale_df.team_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Home"])].attendance)
    scale_df.loc[(scale_df.team_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Home"]),"stad_capac"] = scale_df[(scale_df.team_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Home"])].attendance.apply(lambda x: x/max_attend)
    scale_df.loc[(scale_df.opponent_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Away"]),"stad_capac"] = scale_df[(scale_df.opponent_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Away"])].attendance.apply(lambda x: x/max_attend)

In [6]:
#scale_df = new_data_test.data_frame.copy()

liste = ['schedule_time', 'schedule_round', 'schedule_day', 'result', 'gf', 'ga', 'xg', 'xga', 'formation', 
         'referee', 'season_str', 'league_id', 'team_id', 'opponent_id', 'match_id', 'id', 'fbref_id', 
         'home_team_id', 'away_team_id', 'schedule_date', 'venue', 'captain',]

cols_to_scale = list(set(list(scale_df.columns)).difference(liste))
object_ = StandardScaler()

scale_df.loc[:,cols_to_scale] = object_.fit_transform(scale_df.loc[:,cols_to_scale])
scale_df.head()

,venue,result,gf,ga,xg,xga,attendance,captain,formation,referee,...,misc_aerialduels_lost,misc_aerialduels_won_perc,schedule_date,schedule_time,schedule_round,schedule_day,annual_wage_team,weekly_wages_eur,annual_wage_player_avg,stad_capac
0,1,1,2,3,1.4,1.1,-0.216427,547,2,1,...,-1.837538,2.666886,2023-06-04,1,38,0,-0.875627,-0.875628,-0.649831,0.900104
1,1,2,1,1,0.6,1.2,1.303480,924,3,2,...,-0.842711,-1.995816,2023-06-04,1,38,0,-0.272555,-0.272555,0.368518,0.909555
2,0,1,1,3,0.8,2.4,2.272004,639,1,3,...,-0.700593,0.391488,2023-06-04,1,38,0,-0.759376,-0.759377,-0.473136,0.678084
3,1,1,0,1,0.8,1.3,-0.353520,774,0,4,...,-0.984829,0.661925,2023-06-04,1,38,0,-0.794992,-0.794992,-0.510622,0.722744
4,0,1,2,5,1.2,4.0,-0.561956,548,0,5,...,-0.984829,0.466091,2023-06-04,1,38,0,-0.389500,-0.389500,-0.097483,0.701977


In [7]:
ohe_team = OneHotEncoder()
to_ohe_team = scale_df.loc[:, ["team_id", "opponent_id"]]
ohe_team.fit(to_ohe_team)

codes = ohe_team.transform(to_ohe_team).toarray()
feature_names = ohe_team.get_feature_names(['team_id', 'opponent_id'])

scale_df = pd.concat([scale_df, 
               pd.DataFrame(codes, columns = feature_names).astype(int)], axis=1)
##########################################
ohe_ligue = OneHotEncoder()
to_ohe_ligue = scale_df.loc[:,["league_id"]]
ohe_ligue.fit(to_ohe_ligue)

codes = ohe_ligue.transform(to_ohe_ligue).toarray()
feature_names = ohe_ligue.get_feature_names(['league_id'])

scale_df = pd.concat([scale_df, 
               pd.DataFrame(codes, columns = feature_names).astype(int)], axis=1)


In [8]:
scale_df

,venue,result,gf,ga,xg,xga,attendance,captain,formation,referee,...,opponent_id_139,opponent_id_140,opponent_id_141,opponent_id_142,opponent_id_143,league_id_0,league_id_1,league_id_2,league_id_3,league_id_4
0,1,1,2,3,1.4,1.1,-0.216427,547,2,1,...,0,0,0,0,0,1,0,0,0,0
1,1,2,1,1,0.6,1.2,1.303480,924,3,2,...,0,0,0,0,0,0,1,0,0,0
2,0,1,1,3,0.8,2.4,2.272004,639,1,3,...,0,0,0,0,0,1,0,0,0,0
3,1,1,0,1,0.8,1.3,-0.353520,774,0,4,...,0,0,0,0,0,1,0,0,0,0
4,0,1,2,5,1.2,4.0,-0.561956,548,0,5,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21701,1,1,1,3,0.6,1.9,-0.819691,44,3,168,...,0,0,0,0,0,0,0,1,0,0
21702,1,0,2,0,2.6,0.3,0.916985,126,2,142,...,0,0,0,0,0,0,0,1,0,0
21703,0,1,0,2,0.3,2.6,0.916985,681,10,142,...,0,0,0,0,0,0,0,1,0,0
21704,1,0,3,2,2.0,0.3,-0.874689,391,4,26,...,0,0,0,0,0,0,0,1,0,0


In [9]:
def save_object(obj, save_name):
        """
        Saves obj to a pickle file in directory path.
        """ 
        with open(os.path.join(f"{save_name}.pkl"), 'wb') as f:
            pkl.dump(obj, f)

def load_data_prep_object(obj_name):
    """
    Loads and returns the specified data prep object from the data_prep_objects_path.
    """
        # check if file exists
    if not os.path.isfile(os.path.join(f"{obj_name}.pkl")):
        raise ValueError(f"Data prep object file '{obj_name}.pkl' does not exist.")
    else:
        with open(os.path.join(f"{obj_name}.pkl"), 'rb') as f:
            obj = pkl.load(f)
            return obj
            
def do_pca(df, perc_var, col_name1 = "shooting_standard_gls", col_name2 = 'misc_aerialduels_won_perc'):
    try:
        pcs_matchstat = load_data_prep_object("./pca_matchstats")
    except:
        x = df.loc[:,col_name1:col_name2].fillna(0)
        pca_matchstat = PCA(n_components = perc_var)
        pcs_matchstat = pca_matchstat.fit_transform(x)
        save_object(pcs_matchstat, "pca_matchstats")
    principal_ms_df = pd.DataFrame(data = pcs_matchstat, columns = [f"feature_{p}" for p in range(pcs_matchstat.shape[1])])
    num_pcs = principal_ms_df.shape[1]
    print(num_pcs)
    columns_to_overwrite = list(df.loc[:, col_name1:col_name2].columns)
    df = df.drop(labels = columns_to_overwrite, axis = "columns")
    new_cols = columns_to_overwrite[:num_pcs-1] + columns_to_overwrite[-1:]
    print(len(new_cols))
    df.loc[:, new_cols] = principal_ms_df.values
    return df

In [10]:
scale_df_pca = do_pca(scale_df, 0.95)

66
66


In [11]:
clubs = club_dict(scale_df_pca)

In [12]:
result_dict = new_data_test.return_dicts("result")
clubs = points_and_co(clubs, result_dict)

In [13]:
clubs = points_and_co_oppon(clubs, result_dict)

In [14]:
clubs[12]

,venue,result,gf,ga,xg,xga,attendance,captain,formation,referee,...,passing_types_passtypes_fk,passing_types_passtypes_tb,misc_aerialduels_won_perc,goal_diff,points,mean_points,last_results,oppon_points,oppon_mean_points,oppon_wages
0,1,0,4,1,1.5,0.5,1.714491,772,0,44,...,0.518642,0.067527,1.092243,3,-1.612125,1.575210,-0.968227,0.008358,0.003508,-0.330974
1,0,2,2,2,1.0,1.6,0.543231,772,0,38,...,-0.235132,0.665313,0.215343,0,-1.561622,0.292398,-0.968227,-1.676019,1.654948,1.970107
2,1,0,1,0,1.7,0.7,1.691158,772,0,36,...,-0.073981,0.875204,-0.104222,1,-1.410111,0.933804,-0.968227,-1.549072,-1.097851,-0.280787
3,0,0,2,0,2.4,0.9,-0.029335,772,0,88,...,0.907035,-0.725673,-0.132336,2,-1.258601,1.254507,-0.968227,-1.181903,0.401959,-0.447278
4,0,2,1,1,0.6,1.2,1.753092,772,0,75,...,-0.212094,0.134320,-0.618512,0,-1.208097,0.132047,-0.968227,-1.356236,0.316821,0.264765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,1,1,1,3,0.4,2.9,2.740217,772,3,137,...,-0.474851,-0.105241,0.517217,-2,1.771610,-0.355463,1.016094,1.546557,-1.475060,1.848583
224,0,2,1,1,1.3,1.1,0.032169,772,3,139,...,0.237615,-0.467468,-0.312579,0,1.822113,-0.672938,0.023934,0.858862,-1.146203,-0.203189
225,1,0,2,0,1.4,1.3,1.227568,772,3,72,...,1.133570,-0.857446,-0.195712,2,1.973624,0.451136,0.520014,1.409651,-0.517892,-0.082442
226,0,1,0,1,1.2,1.6,-0.330994,772,0,200,...,-0.770269,1.477042,-1.496773,-1,1.973624,-0.911045,-0.968227,1.801056,2.008802,0.148828


In [15]:
game_train, game_valid = game_dict(scale_df_pca)

In [16]:
abcdefg = list(scale_df.columns)
abc = abcdefg[:abcdefg.index("annual_wage_player_avg")+1]
defg = abcdefg[abcdefg.index("annual_wage_player_avg")+1:]

rearange_list = ['result', 'gf', 'ga', 'goal_diff', 'xg', 'xga', 'shooting_standard_gls',
       'shooting_standard_sh', 'shooting_standard_sot',
       'shooting_standard_sot_perc', 'shooting_standard_g_per_sh',
       'shooting_standard_g_per_sot', 'shooting_standard_dist',
       'shooting_standard_fk', 'shooting_standard_pk',
       'shooting_standard_pkatt', 'shooting_expected_npxg',
       'shooting_expected_npxg_per_sh', 'shooting_expected_g_minus_xg',
       'shooting_expected_npg_minus_xg', 'keeper_performance_sota',
       'keeper_performance_saves', 'keeper_performance_save_perc',
       'keeper_performance_cs', 'keeper_performance_psxg',
       'keeper_performance_psxg_plus_minus', 'keeper_penaltykicks_pkatt',
       'keeper_penaltykicks_pka', 'keeper_penaltykicks_pksv',
       'keeper_penaltykicks_pkm', 'keeper_launched_cmp', 'keeper_launched_att',
       'keeper_launched_cmp_perc', 'keeper_passes_att', 'keeper_passes_thr',
       'keeper_passes_launch_perc', 'keeper_passes_avglen',
       'keeper_goalkicks_att', 'keeper_goalkicks_launch_perc',
       'keeper_goalkicks_avglen', 'keeper_crosses_opp', 'keeper_crosses_stp',
       'keeper_crosses_stp_perc', 'keeper_sweeper_number_opa',
       'keeper_sweeper_avgdist', 'passing_total_cmp', 'passing_total_att',
       'passing_total_cmp_perc', 'passing_total_totdist',
       'passing_total_prgdist', 'passing_short_cmp', 'passing_short_att',
       'passing_short_cmp_perc', 'passing_medium_cmp', 'passing_medium_att',
       'passing_medium_cmp_perc', 'passing_long_cmp', 'passing_long_att',
       'passing_long_cmp_perc', 'passing_attacking_ast',
       'passing_attacking_xag', 'passing_attacking_xa', 'passing_attacking_kp',
       'passing_attacking_1_per_3', 'passing_attacking_ppa',
       'passing_attacking_crspa', 'passing_attacking_prgp',
       'passing_types_passtypes_live', 'passing_types_passtypes_dead',
       'passing_types_passtypes_fk', 'passing_types_passtypes_tb',
       'misc_aerialduels_won_perc','attendance', 'points', 'mean_points',
       'weekly_wages_eur', 'season_str',  'league_id', 'venue', 'team_id', "oppon_wages",
       'opponent_id', 'last_results', 'oppon_points', 'oppon_mean_points', 'schedule_round',
        'captain', 'formation', 'referee',  'match_id', 'schedule_date', 'schedule_time',
        'schedule_day', 'annual_wage_team', 'annual_wage_player_avg',]

rearange_list = list(itertools.chain.from_iterable(defg if item == "team_id" else [item] for item in rearange_list))

del rearange_list[rearange_list.index("opponent_id")]
rearange_list

['result',
 'gf',
 'ga',
 'goal_diff',
 'xg',
 'xga',
 'shooting_standard_gls',
 'shooting_standard_sh',
 'shooting_standard_sot',
 'shooting_standard_sot_perc',
 'shooting_standard_g_per_sh',
 'shooting_standard_g_per_sot',
 'shooting_standard_dist',
 'shooting_standard_fk',
 'shooting_standard_pk',
 'shooting_standard_pkatt',
 'shooting_expected_npxg',
 'shooting_expected_npxg_per_sh',
 'shooting_expected_g_minus_xg',
 'shooting_expected_npg_minus_xg',
 'keeper_performance_sota',
 'keeper_performance_saves',
 'keeper_performance_save_perc',
 'keeper_performance_cs',
 'keeper_performance_psxg',
 'keeper_performance_psxg_plus_minus',
 'keeper_penaltykicks_pkatt',
 'keeper_penaltykicks_pka',
 'keeper_penaltykicks_pksv',
 'keeper_penaltykicks_pkm',
 'keeper_launched_cmp',
 'keeper_launched_att',
 'keeper_launched_cmp_perc',
 'keeper_passes_att',
 'keeper_passes_thr',
 'keeper_passes_launch_perc',
 'keeper_passes_avglen',
 'keeper_goalkicks_att',
 'keeper_goalkicks_launch_perc',
 'keeper_

In [17]:
train_inputs = inputs(game_train, clubs, rearange_list, scale_df_pca)
valid_inputs = inputs(game_valid, clubs, rearange_list, scale_df_pca)

7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [18]:
train_for_loader = data_to_lstm(train_inputs)
valid_for_loader = data_to_lstm(valid_inputs)

In [19]:
train_loader = torch.utils.data.DataLoader(train_for_loader, batch_size = 32, drop_last = True)
test_loader = torch.utils.data.DataLoader(valid_for_loader, batch_size = 32, drop_last = True)

In [20]:
len_input = train_inputs[0][0].shape[1]
lr = 2e-5
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
#print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = lr)#, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred = net(input1)
    #print(pred.shape)
    #print(result1.shape)
    pred = torch.argmax(pred[:,-1,:], dim = 1)
    result = result1[:,-1,:]
    result = torch.argmax(result, dim = 1)
    accur += pred.eq(result).sum().item()

    pred = net(input2)
    pred = torch.argmax(pred[:,-1,:], dim = 1)
    result2 = result2[:,-1,:]
    result2 = torch.argmax(result2, dim = 1)
    accur += pred.eq(result2).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 64)}%")

for epoch in range(5):
    losses_val = []
    accuracies = []

    net.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net.zero_grad()
        pred = net(input1)
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
        #print(f"pred {pred.dtype}, result {result.dtype}")
        #result1 = result1[:,-1,:]
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()

        net.zero_grad()
        pred = net(input2)
        #result2 = result2[:,-1,:]
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
        #print(f"pred {pred.dtype}, result {result.dtype}")
        loss = criterion(pred.float(), result2.float())
        loss.backward()
        optimizer.step()

    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred = net(input1)
        #print(pred)
        #print(pred.shape)
        result1 = result1[:,-1,:]
        pred = pred[:,-1,:]
        loss += criterion(pred, result1).item()
        #
        #print(pred)
        #print(pred.shape)
        pred = torch.argmax(pred, dim = 1)
        #print(pred)
        #print(pred.shape)

        result1 = torch.argmax(result1, dim = 1)
        accur += pred.eq(result1).sum().item()
       # if epoch == 9:
        #    print("pred", pred)
         #   print("result", result1)
        pred = net(input2)
        result2 = result2[:,-1,:]
        pred = pred[:,-1,:]
        loss += criterion(pred, result2).item()
        #pred = pred[:,-1,:]
        pred = torch.argmax(pred, dim = 1)
        result2  = torch.argmax(result2, dim = 1)
        accur += pred.eq(result2).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 64))
    os.makedirs(os.path.dirname(f"./models/sequence_model_pca/LSTM/{lr}/accur_{round(accuracy, 2)}"), exist_ok = True)
    torch.save(net.state_dict(), f"./models/sequence_model_pca/LSTM/{lr}/accur_{round(accuracy, 2)}")
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 
print("done")


Loss accuracy training: 24.762228260869566%
Epoch: 0
Loss 38.54323095083237 accuracy 46.73913043478261%
Epoch: 1
Loss 37.75224082171917 accuracy 48.77717391304348%
Epoch: 2
Loss 38.299265936017036 accuracy 48.74320652173913%
Epoch: 3
Loss 40.73889720439911 accuracy 47.14673913043478%
Epoch: 4
Loss 45.714231580495834 accuracy 45.21059782608695%
done


In [50]:
len_input = train_inputs[0][0].shape[1]
net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
net.load_state_dict(torch.load("./models/sequence_model_pca/LSTM/2e-05/accur_49.15"))

<All keys matched successfully>

In [52]:
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred1 = net(input1)[:,-1,:]
    pred2 = net(input2)[:,-1,:]
    pred2 = torch.index_select(pred2, 1, torch.LongTensor([1,0,2]))
    pred1_ana = pred1
    pred2_ana = pred2
    pred1 = torch.argmax(pred1, dim = 1)
    pred2 = torch.argmax(pred2, dim = 1)
    index = pred1.ne(pred2)
    result = result1[index,-1,:]
    result = torch.argmax(result)
    pred = pred1[index]
    accur += pred.eq(result).sum().item()
accuracy = 100 * (accur /((step + 1) * 32))
print(f"{accuracy}%")

18.44429347826087%


In [49]:
pred2 = net(input2)[:,-1,:]
pred2 = torch.index_select(pred2, 1, torch.LongTensor([1,0,2]))
pred2

tensor([[0.3885, 0.3979, 0.2049],
        [0.4149, 0.3292, 0.2163],
        [0.3974, 0.3785, 0.2076],
        [0.5119, 0.3076, 0.1876],
        [0.3974, 0.3785, 0.2076],
        [0.4508, 0.3424, 0.2161],
        [0.5119, 0.3076, 0.1876],
        [0.4508, 0.3424, 0.2161],
        [0.4149, 0.3292, 0.2163],
        [0.5010, 0.1752, 0.2528],
        [0.4449, 0.3201, 0.1936],
        [0.3034, 0.4918, 0.1590],
        [0.4757, 0.1892, 0.2909],
        [0.2674, 0.5076, 0.1830],
        [0.4757, 0.1892, 0.2909],
        [0.2674, 0.5076, 0.1830],
        [0.4169, 0.2666, 0.2844],
        [0.3058, 0.4845, 0.2259],
        [0.3905, 0.1333, 0.3938],
        [0.3058, 0.4845, 0.2259],
        [0.4269, 0.3050, 0.2648],
        [0.4169, 0.2666, 0.2844],
        [0.3905, 0.1333, 0.3938],
        [0.4836, 0.3002, 0.2262],
        [0.4113, 0.3396, 0.2519],
        [0.4269, 0.3050, 0.2648],
        [0.4113, 0.3396, 0.2519],
        [0.4836, 0.3002, 0.2262],
        [0.4561, 0.3066, 0.2311],
        [0.456

In [38]:
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred = net(input1)
    #print(pred)
    #print(pred.shape)
    result1 = result1[:,-1,:]
    pred = pred[:,-1,:]
    loss += criterion(pred, result1).item()
    #
    #print(pred)
    #print(pred.shape)
    pred = torch.argmax(pred, dim = 1)
    #print(pred)
    #print(pred.shape)

    result1 = torch.argmax(result1, dim = 1)
    accur += pred.eq(result1).sum().item()
   # if epoch == 9:
    #    print("pred", pred)
     #   print("result", result1)
    pred = net(input2)
    result2 = result2[:,-1,:]
    pred = pred[:,-1,:]
    loss += criterion(pred, result2).item()
    #pred = pred[:,-1,:]
    pred = torch.argmax(pred, dim = 1)
    result2  = torch.argmax(result2, dim = 1)
    accur += pred.eq(result2).sum().item()
losses_val.append(loss)
accuracy = 100 * (accur /((step + 1) * 64))
print(f"{accuracy}%")

49.252717391304344%


In [20]:
class Pred_result(torch.nn.Module):
    def __init__(self, n_hidden, n_classes, activation, softmax = 1):
        super(Pred_result, self).__init__()
        self.n_hidden = n_hidden
        self.n_classes = n_classes
        acties = {"relu": torch.nn.ReLU(), "sigmoid": torch.nn.Sigmoid(), "tanh": torch.nn.Tanh()}
        self.act = acties[activation]
        
        self.first_layer = torch.nn.Linear(6, self.n_hidden)
        self.second_layer = torch.nn.Linear(self.n_hidden, self.n_classes)
        out = {1: torch.nn.Softmax(), 0: torch.nn.Identity()}
        self.out = out[softmax]
    def forward(self, x, y):
        input_ = torch.cat([x, y], dim = 1)
        #print(input_.shape)
        lay1 = self.act(self.first_layer(input_))
        out = self.out(self.act(self.second_layer(lay1)))
        return out

In [25]:
len_input = train_inputs[0][0].shape[1]
lr = 1e-5
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
lrs = [1e-3, 1e-4, 1e-5, 1e-6]
for lr in lrs:
    for activ in ["relu", "sigmoid", "tanh"]:
        for i in [1,0]:
            print(f"{lr} mit {activ} und softmax {i}")
            net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
            net2 = Pred_result(12, 3, activ, i)
            #print(f"\n\nLR: {lr}, WD: {wd}")
            criterion = torch.nn.MSELoss()
            optimizer = torch.optim.Adam(net2.parameters(), lr = lr)#, weight_decay = 0.01)

            net.load_state_dict(torch.load("./models/sequence_model_pca/LSTM/2e-05/accur_49.15"))
            net.eval()

            net2.eval()
            accur = 0
            for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
                pred1 = net(input1)
                pred2 = net(input2)
                pred1 = pred1[:,-1,:]
                pred2 = pred2[:,-1,:]
                #print(pred1.shape)
                #print(pred2.shape)
                result = result1[:,-1,:]
                result = torch.argmax(result, dim = 1)
                predict = net2(pred1, pred2)
                predict = torch.argmax(predict, dim = 1)
                accur += predict.eq(result).sum().item()

            print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")

            for epoch in range(10):
                losses_val = []
                accuracies = []

                net2.train()
                print(f"Epoch: {epoch}")
                for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
                    net2.zero_grad()
                    pred1 = net(input1)[:,-1,:]
                    pred2 = net(input2)[:,-1,:]
                    predict = net2(pred1, pred2)
                    result1 = result1[:,-1,:]
                    loss = criterion(predict.float(), result1.float())
                    loss.backward()
                    optimizer.step()


                net2.eval()
                loss = 0
                accur = 0
                for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
                    pred1 = net(input1)
                    pred2 = net(input2)
                    pred1 = pred1[:,-1,:]
                    pred2 = pred2[:,-1,:]
                    result = result1[:,-1,:]
                    result = torch.argmax(result, dim = 1)
                    predict = net2(pred1, pred2)
                    predict = torch.argmax(predict, dim = 1)
                    accur += predict.eq(result).sum().item()
                    loss += criterion(predict.float(), result.float()).item()
                losses_val.append(loss)
                accuracy = 100 * (accur /((step + 1) * 32))
                #os.makedirs(os.path.dirname(f"./models/sequence_model_pca/LSTM/{lr}/accur_{round(accuracy, 2)}"), exist_ok = True)
                #torch.save(net.state_dict(), f"./models/sequence_model_pca/LSTM/{lr}/accur_{round(accuracy, 2)}")
                accuracies.append(accuracy)
                print(f"Loss {loss} accuracy {accuracy}%") 
            print("done")


0.001 mit relu und softmax 1
Loss accuracy training: 25.407608695652176%
Epoch: 0
Loss 128.125 accuracy 36.95652173913043%
Epoch: 1
Loss 128.125 accuracy 36.95652173913043%
Epoch: 2
Loss 128.125 accuracy 36.95652173913043%
Epoch: 3
Loss 128.125 accuracy 36.95652173913043%
Epoch: 4
Loss 128.125 accuracy 36.95652173913043%
Epoch: 5
Loss 128.125 accuracy 36.95652173913043%
Epoch: 6
Loss 128.125 accuracy 36.95652173913043%
Epoch: 7
Loss 128.125 accuracy 36.95652173913043%
Epoch: 8
Loss 128.125 accuracy 36.95652173913043%
Epoch: 9
Loss 128.125 accuracy 36.95652173913043%
done
0.001 mit relu und softmax 0
Loss accuracy training: 28.464673913043477%
Epoch: 0
Loss 57.75 accuracy 37.63586956521739%
Epoch: 1
Loss 66.0625 accuracy 38.17934782608695%
Epoch: 2
Loss 70.8125 accuracy 38.51902173913043%
Epoch: 3
Loss 71.8125 accuracy 38.45108695652174%
Epoch: 4
Loss 71.0625 accuracy 38.45108695652174%
Epoch: 5
Loss 70.625 accuracy 38.72282608695652%
Epoch: 6
Loss 68.9375 accuracy 38.315217391304344%
E

Loss 57.375 accuracy 37.63586956521739%
Epoch: 3
Loss 57.375 accuracy 37.63586956521739%
Epoch: 4
Loss 57.375 accuracy 37.63586956521739%
Epoch: 5
Loss 57.375 accuracy 37.63586956521739%
Epoch: 6
Loss 57.375 accuracy 37.63586956521739%
Epoch: 7
Loss 57.375 accuracy 37.63586956521739%
Epoch: 8
Loss 57.375 accuracy 37.63586956521739%
Epoch: 9
Loss 57.375 accuracy 37.63586956521739%
done
1e-05 mit sigmoid und softmax 0
Loss accuracy training: 36.95652173913044%
Epoch: 0
Loss 128.125 accuracy 36.95652173913043%
Epoch: 1
Loss 128.125 accuracy 36.95652173913043%
Epoch: 2
Loss 128.125 accuracy 36.95652173913043%
Epoch: 3
Loss 128.125 accuracy 36.95652173913043%
Epoch: 4
Loss 128.125 accuracy 36.95652173913043%
Epoch: 5
Loss 128.125 accuracy 36.95652173913043%
Epoch: 6
Loss 128.125 accuracy 36.95652173913043%
Epoch: 7
Loss 128.125 accuracy 36.95652173913043%
Epoch: 8
Loss 128.125 accuracy 36.95652173913043%
Epoch: 9
Loss 128.125 accuracy 36.95652173913043%
done
1e-05 mit tanh und softmax 1
Los

KeyboardInterrupt: 

In [ ]:
1e-05 mit tanh und softmax 0
0.0001 mit tanh und softmax 0
0.0001 mit tanh und softmax 1
0.0001 mit sigmoid und softmax 0
0.0001 mit sigmoid und softmax 1
0.001 mit tanh und softmax 0
0.001 mit tanh und softmax 1
0.001 mit sigmoid und softmax 0
0.001 mit sigmoid und softmax 1

In [58]:
#len_input = train_inputs[0][0].shape[1]
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
#net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
#print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 2e-5)#, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred1 = net(input1)[:,-1,:]
    pred2 = net(input2)[:,-1,:]
    pred1 = torch.argmax(pred1, dim = 1)
    pred2 = torch.argmax(pred2, dim = 1)
    accur += pred1.eq(pred2).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")


net.eval()
accur = 0
accur = 0
all_wrong = torch.empty((0,6))
all_wrong_res = torch.empty((0,3))
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred1 = net(input1)[:,-1,:]
    pred2 = net(input2)[:,-1,:]
    pred2 = torch.index_select(pred2, 1, torch.LongTensor([1,0,2]))
    pred1_ana = torch.nn.functional.softmax(pred1)
    pred2_ana = torch.nn.functional.softmax(pred2)
    pred1 = torch.argmax(pred1, dim = 1)
    pred2 = torch.argmax(pred2, dim = 1)
    accur += pred1.eq(pred2).sum().item()
    index = pred1.ne(pred2)
    wrong = torch.cat([pred1_ana[index,:], pred2_ana[index,:]], dim = 1)
    all_wrong = torch.cat([all_wrong, wrong], dim = 0)
    wrong_res = result1[index,-1,:]
    all_wrong_res = torch.cat([all_wrong_res, wrong_res], dim = 0)
losses_val.append(loss)
accuracy = 100 * (accur /((step + 1) * 32))
accuracies.append(accuracy)
print(f"Loss {loss} accuracy {accuracy}%") 
print("done")


Loss accuracy training: 47.48641304347826%
Loss 0 accuracy 50.06793478260869%
done


In [79]:
def good_result(pred1, pred2):
    pred2 = torch.index_select(pred2, 1, torch.LongTensor([1,0,2]))
    pred1 = torch.nn.functional.softmax(pred1)
    pred2 = torch.nn.functional.softmax(pred2)
    max1, ind1 = torch.max(pred1, dim = 1)
    max2, ind2 = torch.max(pred2, dim = 1)
    pred1 = torch.argmax(pred1, dim = 1)
    pred2 = torch.argmax(pred2, dim = 1)
    index = pred1.ne(pred2)
    pred1[index] = torch.where(max1[index] > max2[index], ind1[index], ind2[index])
    return pred1

In [71]:
pred2 = torch.index_select(pred2_ana, 1, torch.LongTensor([1,0,2]))
pred1 = torch.nn.functional.softmax(pred1_ana)
pred2 = torch.nn.functional.softmax(pred2)
max1, ind1 = torch.max(pred1, dim = 1)
max2, ind2 = torch.max(pred2, dim = 1)
pred1 = torch.argmax(pred1, dim = 1)
pred2 = torch.argmax(pred2, dim = 1)
index = pred1.ne(pred2)
pred1[index] = torch.where(max1[index] > max2[index], ind1[index], ind2[index])

In [72]:
pred1

tensor([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1])

In [59]:
for elem in zip(all_wrong, all_wrong_res):
    print(elem)
    

(tensor([0.4106, 0.2948, 0.2945, 0.2759, 0.4185, 0.3057],
       grad_fn=<UnbindBackward0>), tensor([0., 1., 0.]))
(tensor([0.4106, 0.2948, 0.2945, 0.2759, 0.4185, 0.3057],
       grad_fn=<UnbindBackward0>), tensor([0., 1., 0.]))
(tensor([0.4471, 0.2611, 0.2918, 0.2903, 0.3911, 0.3186],
       grad_fn=<UnbindBackward0>), tensor([0., 0., 1.]))
(tensor([0.3117, 0.3523, 0.3360, 0.3588, 0.3277, 0.3135],
       grad_fn=<UnbindBackward0>), tensor([1., 0., 0.]))
(tensor([0.4471, 0.2611, 0.2918, 0.2903, 0.3911, 0.3186],
       grad_fn=<UnbindBackward0>), tensor([0., 0., 1.]))
(tensor([0.3117, 0.3523, 0.3360, 0.3588, 0.3277, 0.3135],
       grad_fn=<UnbindBackward0>), tensor([1., 0., 0.]))
(tensor([0.3528, 0.3448, 0.3024, 0.3153, 0.4065, 0.2782],
       grad_fn=<UnbindBackward0>), tensor([0., 0., 1.]))
(tensor([0.3528, 0.3448, 0.3024, 0.3153, 0.4065, 0.2782],
       grad_fn=<UnbindBackward0>), tensor([0., 0., 1.]))
(tensor([0.3287, 0.3433, 0.3280, 0.3760, 0.3120, 0.3120],
       grad_fn=<Unbind

(tensor([0.2854, 0.4248, 0.2898, 0.3723, 0.3353, 0.2924],
       grad_fn=<UnbindBackward0>), tensor([1., 0., 0.]))
(tensor([0.3975, 0.3172, 0.2853, 0.3105, 0.4090, 0.2805],
       grad_fn=<UnbindBackward0>), tensor([1., 0., 0.]))
(tensor([0.3975, 0.3172, 0.2853, 0.3105, 0.4090, 0.2805],
       grad_fn=<UnbindBackward0>), tensor([1., 0., 0.]))
(tensor([0.2854, 0.4248, 0.2898, 0.3723, 0.3353, 0.2924],
       grad_fn=<UnbindBackward0>), tensor([1., 0., 0.]))
(tensor([0.3116, 0.3867, 0.3017, 0.3544, 0.3495, 0.2962],
       grad_fn=<UnbindBackward0>), tensor([0., 1., 0.]))
(tensor([0.3116, 0.3867, 0.3017, 0.3544, 0.3495, 0.2962],
       grad_fn=<UnbindBackward0>), tensor([0., 1., 0.]))
(tensor([0.2630, 0.4298, 0.3072, 0.3626, 0.3423, 0.2951],
       grad_fn=<UnbindBackward0>), tensor([0., 1., 0.]))
(tensor([0.2630, 0.4298, 0.3072, 0.3626, 0.3423, 0.2951],
       grad_fn=<UnbindBackward0>), tensor([0., 1., 0.]))
(tensor([0.2883, 0.4051, 0.3066, 0.4103, 0.2868, 0.3029],
       grad_fn=<Unbind

In [80]:
len_input = train_inputs[0][0].shape[1]
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
#print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 2e-5)#, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred1 = net(input1)[:,-1,:]
    pred2 = net(input2)[:,-1,:]
    pred1 = torch.argmax(pred1, dim = 1)
    pred2 = torch.argmax(pred2, dim = 1)
    accur += pred1.eq(pred2).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")

for epoch in range(1):  
    
    losses_val = []
    accuracies = []

#    net.train()
    print(f"Epoch: {epoch}")
    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred1 = net(input1)[:,-1,:]
        pred2 = net(input2)[:,-1,:]
        pred = good_result(pred1, pred2)
        result = result1[:,-1,:]
        result = torch.argmax(result, dim = 1)
        accur += pred.eq(result).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 32))
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 
print("done")


Loss accuracy training: 91.44021739130434%
Epoch: 0
Loss 0 accuracy 35.733695652173914%
done


In [62]:
for elem in zip(all_wrong, all_wrong_res):
    print(elem)

(tensor([-0.0014, -0.0043, -0.0575, -0.0116, -0.0025, -0.0495],
       grad_fn=<UnbindBackward0>), tensor([0., 0., 1.]))
(tensor([-0.0014, -0.0043, -0.0575, -0.0116, -0.0025, -0.0495],
       grad_fn=<UnbindBackward0>), tensor([0., 0., 1.]))
(tensor([-0.0180, -0.0171, -0.0503, -0.0060, -0.0244, -0.0277],
       grad_fn=<UnbindBackward0>), tensor([0., 1., 0.]))
(tensor([-0.0180, -0.0171, -0.0503, -0.0060, -0.0244, -0.0277],
       grad_fn=<UnbindBackward0>), tensor([0., 1., 0.]))
(tensor([ 0.0068, -0.0092, -0.0457, -0.0092, -0.0032, -0.0701],
       grad_fn=<UnbindBackward0>), tensor([0., 0., 1.]))
(tensor([ 0.0068, -0.0092, -0.0457, -0.0092, -0.0032, -0.0701],
       grad_fn=<UnbindBackward0>), tensor([0., 0., 1.]))
(tensor([-0.0238, -0.0104, -0.0572,  0.0011, -0.0095, -0.0308],
       grad_fn=<UnbindBackward0>), tensor([1., 0., 0.]))
(tensor([-0.0238, -0.0104, -0.0572,  0.0011, -0.0095, -0.0308],
       grad_fn=<UnbindBackward0>), tensor([1., 0., 0.]))
(tensor([-0.0083, -0.0233, -0.03

In [63]:
torch.sum(all_wrong_res, dim = 0)

tensor([288., 246., 210.])

In [48]:
torch.empty((0,3))

tensor([], size=(0, 3))

In [21]:
len_input = train_inputs[0][0].shape[1]
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
#print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 2e-5)#, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred1 = net(input1)
    pred2 = net(input2)
    result = result1[:,-1,:]
    result = torch.argmax(result, dim = 1)
    predicted = predict(pred1, pred2, result_dict)
    accur += predicted.eq(result).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")

for epoch in range(10):
    losses_val = []
    accuracies = []

    net.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net.zero_grad()
        pred = net(input1)
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()

        net.zero_grad()
        pred = net(input2)
        loss = criterion(pred.float(), result2.float())
        loss.backward()
        optimizer.step()

    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred1 = net(input1)
        pred2 = net(input2)
        result = result1[:,-1,:]
        result = torch.argmax(result, dim = 1)
        predicted = predict(pred1, pred2, result_dict)
        #predicted = torch.nn.functional.one_hot(predict(pred1, pred2, result_dict), num_classes = 3)
        accur += predicted.eq(result).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 32))
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 
print("done")


Loss accuracy training: 36.005434782608695%
Epoch: 0
Loss 0 accuracy 44.42934782608695%
Epoch: 1
Loss 0 accuracy 44.565217391304344%
Epoch: 2
Loss 0 accuracy 43.953804347826086%
Epoch: 3


KeyboardInterrupt: 

In [ ]:
result

In [ ]:
predicted

In [ ]:
def predict(tensor1, tensor2, result_dict):
    # tensor with results
    tens1 = tensor1[:,-1,:]
    tens2 = tensor2[:,-1,:]
    
    sum1 = torch.sum(tens1, dim = 1)
    sum2 = torch.sum(tens2, dim = 1)
    sum1_reshaped = sum1.unsqueeze(1).expand_as(tens1)
    sum2_reshaped = sum2.unsqueeze(1).expand_as(tens2)
    output1 = tens1 / sum1_reshaped
    output2 = tens2 / sum1_reshaped
    
    vals1, idx1 = torch.max(output1, dim = 1)
    vals2, idx2 = torch.max(output2, dim = 1)
    #high_val1 = torch.max(output1, dim = 1)
    #high_val2 = torch.argmax(output2, dim = 1)
    #print(high_val1.shape)
    small_val1,_ = torch.topk(output1, k = 2, dim = 1, largest = False)
    small_val2,_ = torch.topk(output2, k = 2, dim = 1, largest = False)
    
    diff1 = torch.sum(torch.square(vals1.unsqueeze(1).expand_as(small_val1) - small_val1), dim = 1)
    diff2 = torch.sum(torch.square(vals2.unsqueeze(1).expand_as(small_val2) - small_val2), dim = 1) 
    
    result = torch.where((diff1 - diff2) > 0.2, idx1, torch.where((diff2 - diff1) > 0.2, idx2, result_dict["D"]))
        
    return result

In [ ]:
result_dict